<a href="https://colab.research.google.com/github/pierre700/BeerGamePRT20232024/blob/main/FSL_Beginning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader, random_split
from torchvision import models, transforms
import cv2
from PIL import Image
import os

# Define the image transformations
transform = transforms.Compose([
    transforms.ToPILImage(),
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    
])

# Define the Siamese Network
class SiameseNetwork(nn.Module):
    def __init__(self):
        super(SiameseNetwork, self).__init__()
        self.resnet = models.resnet18(weights='IMAGENET1K_V1')
        
        # Freeze all layers except the classifier
        for param in self.resnet.parameters():
            param.requires_grad = False
        
        # Modify the fully connected layer
        self.resnet.fc = nn.Linear(self.resnet.fc.in_features, 512)

    def forward_one(self, x):
        x = self.resnet(x)
        return x

    def forward(self, input1, input2):
        output1 = self.forward_one(input1)
        output2 = self.forward_one(input2)
        return output1, output2

# Define the dataset for image pairs
class ImagePairDataset(Dataset):
    def __init__(self, image_dir, shots_dir, transform=None):
        self.image_dir = image_dir
        self.shots_dir = shots_dir
        self.transform = transform
        self.image_pairs = []
        self.labels = []
        self._prepare_data()

    def _prepare_data(self):
        shots_files = os.listdir(self.shots_dir)
        image_files = os.listdir(self.image_dir)
        print(f"Number of shots: {len(shots_files)}")
        print(f"Number of images in anonymized dataset: {len(image_files)}")
        for shot in shots_files:
            for img in image_files:
                shot_path = os.path.join(self.shots_dir, shot)
                img_path = os.path.join(self.image_dir, img)
                # Example condition for labeling, modify as needed
                label = 1 if shot.split('_')[0] == img.split('_')[0] else 0
                self.image_pairs.append((shot_path, img_path))
                self.labels.append(label)
               


        print(f"Total image pairs created: {len(self.image_pairs)}")

    def __len__(self):
        return len(self.image_pairs)

    def __getitem__(self, idx):
        image1_path, image2_path = self.image_pairs[idx]
        image1 = cv2.imread(image1_path)
        image2 = cv2.imread(image2_path)
        image1 = cv2.cvtColor(image1, cv2.COLOR_BGR2RGB)
        image2 = cv2.cvtColor(image2, cv2.COLOR_BGR2RGB)

        if self.transform:
            image1 = self.transform(image1)
            image2 = self.transform(image2)

        label = torch.tensor(self.labels[idx], dtype=torch.float32)
        return image1, image2, label



# Function to compute the Euclidean distance
def euclidean_distance(a, b):
    return F.pairwise_distance(a, b)

# Training function
def train(model, dataloader, criterion, optimizer, device):
    model.train()
    running_loss = 0.0
    for image1, image2, label in dataloader:
        image1, image2, label = image1.to(device), image2.to(device), label.to(device)
        optimizer.zero_grad()
        output1, output2 = model(image1, image2)
        loss = criterion(euclidean_distance(output1, output2), label)
        loss.backward()
        optimizer.step()
        running_loss += loss.item()
    return running_loss / len(dataloader)

# Evaluation function
def evaluate(model, dataloader, threshold, device):
    model.eval()
    correct = 0
    total = 0
    with torch.no_grad():
        for image1, image2, label in dataloader:
            image1, image2, label = image1.to(device), image2.to(device), label.to(device)
            output1, output2 = model(image1, image2)
            distance = euclidean_distance(output1, output2)
            predictions = (distance < threshold).float()
            correct += (predictions == label).sum().item()
            total += label.size(0)
    return correct / total

# Function to classify similarity of shots with anonymized dataset
def classify_similarity(model, shots_dir, image_dir, threshold, device):
    model.eval()
    transform = transforms.Compose([
        transforms.ToPILImage(),
        transforms.Resize((224, 224)),
        transforms.ToTensor(),
  
    ])
    
    shots_files = os.listdir(shots_dir)
    image_files = os.listdir(image_dir)
    results = []

    with torch.no_grad():
        for shot in shots_files:
            shot_path = os.path.join(shots_dir, shot)
            shot_image = cv2.imread(shot_path)
            shot_image = cv2.cvtColor(shot_image, cv2.COLOR_BGR2RGB)
            shot_image = transform(shot_image).unsqueeze(0).to(device)
            
            for img in image_files:
                img_path = os.path.join(image_dir, img)
                img_image = cv2.imread(img_path)
                img_image = cv2.cvtColor(img_image, cv2.COLOR_BGR2RGB)
                img_image = transform(img_image).unsqueeze(0).to(device)
                
                output1, output2 = model(shot_image, img_image)
                distance = euclidean_distance(output1, output2).item()
                if distance < threshold:
                    results.append((shot, img, distance))
                    print(f"Shot: {shot}, Image: {img}, Distance: {distance}, Similar: True")
                else:
                    print(f"Shot: {shot}, Image: {img}, Distance: {distance}, Similar: False")

    return results

# Main function
def main(image_dir, shots_dir, batch_size=16, num_epochs=10, learning_rate=0.001, threshold=0.5):
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    dataset = ImagePairDataset(image_dir, shots_dir, transform=transform)

    if len(dataset) == 0:
        print("No image pairs found. Please check the dataset paths and image files.")
        return

    train_size = int(0.8 * len(dataset))
    test_size = len(dataset) - train_size

    if train_size == 0 or test_size == 0:
        print(f"Train size: {train_size}, Test size: {test_size}. Not enough data to split.")
        return

    train_dataset, test_dataset = random_split(dataset, [train_size, test_size])
    train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
    test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

    model = SiameseNetwork().to(device)
    criterion = nn.BCEWithLogitsLoss()
    optimizer = torch.optim.Adam(model.resnet.fc.parameters(), lr=learning_rate)

    for epoch in range(num_epochs):
        train_loss = train(model, train_loader, criterion, optimizer, device)
        test_accuracy = evaluate(model, test_loader, threshold, device)
        print(f"Epoch [{epoch+1}/{num_epochs}], Loss: {train_loss:.4f}, Accuracy: {test_accuracy:.4f}")

    print("Training complete")

    # Final test evaluation
    final_test_accuracy = evaluate(model, test_loader, threshold, device)
    print(f"Final Test Accuracy: {final_test_accuracy:.4f}")

    # Classify similarity
    classify_similarity(model, shots_dir, image_dir, threshold, device)

# Example usage
image_dir = '/content/drive/MyDrive/urv/face_recognition/train/Aamir Khan'
shots_dir = '/content/drive/MyDrive/urv/face_recognition/train/Aamir Khan'
main(image_dir, shots_dir)


In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader, random_split
from torchvision import models, transforms
import cv2
import os

# Define the image transformations
transform = transforms.Compose([
    transforms.ToPILImage(),
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
])

# Define the function to get all files from directories and subdirectories
def get_all_files(dir_path):
    all_files = []
    for root, _, files in os.walk(dir_path):
        for file in files:
            if file.endswith(('jpg', 'jpeg', 'png')):
                all_files.append(os.path.join(root, file))
    return all_files

# Define the Siamese Network
class SiameseNetwork(nn.Module):
    def __init__(self):
        super(SiameseNetwork, self).__init__()
        self.resnet = models.resnet18(weights='IMAGENET1K_V1')

        # Freeze all layers except the classifier
        for param in self.resnet.parameters():
            param.requires_grad = False

        # Modify the fully connected layer
        self.resnet.fc = nn.Linear(self.resnet.fc.in_features, 512)

    def forward_one(self, x):
        x = self.resnet(x)
        return x

    def forward(self, input1, input2):
        output1 = self.forward_one(input1)
        output2 = self.forward_one(input2)
        return output1, output2

# Define the dataset for image pairs
class ImagePairDataset(Dataset):
    def __init__(self, image_dir, shots_dir, transform=None):
        self.image_dir = image_dir
        self.shots_dir = shots_dir
        self.transform = transform
        self.image_pairs = []
        self.labels = []
        self._prepare_data()

    def _prepare_data(self):
        shots_files = get_all_files(self.shots_dir)
        image_files = get_all_files(self.image_dir)

        print(f"Number of shots: {len(shots_files)}")
        print(f"Number of images in anonymized dataset: {len(image_files)}")

        for shot_path in shots_files:
            for img_path in image_files:
                shot_id = os.path.basename(os.path.dirname(shot_path))
                img_id = os.path.basename(os.path.dirname(img_path))

                # Label 1 if they are from the same person, otherwise 0
                label = 1 if shot_id == img_id else 0

                self.image_pairs.append((shot_path, img_path))
                self.labels.append(label)

        print(f"Total image pairs created: {len(self.image_pairs)}")

    def __len__(self):
        return len(self.image_pairs)

    def __getitem__(self, idx):
        image1_path, image2_path = self.image_pairs[idx]
        image1 = cv2.imread(image1_path)
        image2 = cv2.imread(image2_path)
        image1 = cv2.cvtColor(image1, cv2.COLOR_BGR2RGB)
        image2 = cv2.cvtColor(image2, cv2.COLOR_BGR2RGB)

        if self.transform:
            image1 = self.transform(image1)
            image2 = self.transform(image2)

        label = torch.tensor(self.labels[idx], dtype=torch.float32)
        return image1, image2, label

# Function to compute the Euclidean distance
def euclidean_distance(a, b):
    return F.pairwise_distance(a, b)

# Training function
def train(model, dataloader, criterion, optimizer, device):
    model.train()
    running_loss = 0.0
    for image1, image2, label in dataloader:
        image1, image2, label = image1.to(device), image2.to(device), label.to(device)
        optimizer.zero_grad()
        output1, output2 = model(image1, image2)
        loss = criterion(euclidean_distance(output1, output2), label)
        loss.backward()
        optimizer.step()
        running_loss += loss.item()
    return running_loss / len(dataloader)

# Evaluation function
def evaluate(model, dataloader, threshold, device):
    model.eval()
    correct = 0
    total = 0
    with torch.no_grad():
        for image1, image2, label in dataloader:
            image1, image2, label = image1.to(device), image2.to(device), label.to(device)
            output1, output2 = model(image1, image2)
            distance = euclidean_distance(output1, output2)
            predictions = (distance < threshold).float()
            correct += (predictions == label).sum().item()
            total += label.size(0)
    return correct / total

# Function to classify similarity of shots with anonymized dataset
def classify_similarity(model, shots_dir, image_dir, threshold, device):
    model.eval()
    transform = transforms.Compose([
        transforms.ToPILImage(),
        transforms.Resize((224, 224)),
        transforms.ToTensor(),
    ])

    shots_files = get_all_files(shots_dir)
    image_files = get_all_files(image_dir)
    results = []

    with torch.no_grad():
        for shot_path in shots_files:
            shot_name = os.path.basename(shot_path)
            shot_image = cv2.imread(shot_path)
            shot_image = cv2.cvtColor(shot_image, cv2.COLOR_BGR2RGB)
            shot_image = transform(shot_image).unsqueeze(0).to(device)

            for img_path in image_files:
                img_name = os.path.basename(img_path)
                img_image = cv2.imread(img_path)
                img_image = cv2.cvtColor(img_image, cv2.COLOR_BGR2RGB)
                img_image = transform(img_image).unsqueeze(0).to(device)
                # label = 1 if shot_image == image_files else 0
                label = 1 if shot_name == img_name else 0
                output1, output2 = model(shot_image, img_image)
                distance = euclidean_distance(output1, output2).item()
                similar = distance < threshold
                results.append((shot_name, img_name, distance, similar))

                # Print shot name, image name, and similarity status
                if similar:
                    print(f"Shot: {shot_name}, Image: {img_name}, Distance: {distance:.4f}, Similar: True, label: {label}")
                else:
                    print(f"Shot: {shot_name}, Image: {img_name}, Distance: {distance:.4f}, Similar: False, label: {label}")

    return results

# Main function
def main(image_dir, shots_dir, batch_size=16, num_epochs=150, learning_rate=0.001, threshold=6.0):
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    dataset = ImagePairDataset(image_dir, shots_dir, transform=transform)

    if len(dataset) == 0:
        print("No image pairs found. Please check the dataset paths and image files.")
        return

    train_size = int(0.8 * len(dataset))
    test_size = len(dataset) - train_size

    if train_size == 0 or test_size == 0:
        print(f"Train size: {train_size}, Test size: {test_size}. Not enough data to split.")
        return

    train_dataset, test_dataset = random_split(dataset, [train_size, test_size])
    train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
    test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

    model = SiameseNetwork().to(device)
    criterion = nn.BCEWithLogitsLoss()
    optimizer = torch.optim.Adam(model.resnet.fc.parameters(), lr=learning_rate)

    for epoch in range(num_epochs):
        train_loss = train(model, train_loader, criterion, optimizer, device)
        test_accuracy = evaluate(model, test_loader, threshold, device)
        print(f"Epoch [{epoch+1}/{num_epochs}], Loss: {train_loss:.4f}, Accuracy: {test_accuracy:.4f}")

    print("Training complete")
    torch.save(model.state_dict(), "/content/drive/MyDrive/urv/distance_model.pt")
    # Final test evaluation
    final_test_accuracy = evaluate(model, test_loader, threshold, device)
    print(f"Final Test Accuracy: {final_test_accuracy:.4f}")

    # Classify similarity
    classify_similarity(model, shots_dir, image_dir, threshold, device)

# Example usage
image_dir = '/content/drive/MyDrive/urv/Images_For_The_Person_i_Want_To_Identify_In_Anonymized_Dataset_with_noise'
shots_dir = '/content/drive/MyDrive/urv/Images_For_The_Person_i_Want_To_Identify_In_Anonymized_Dataset_without_noise'
main(image_dir, shots_dir)


Number of shots: 26
Number of images in anonymized dataset: 26
Total image pairs created: 676
Epoch [1/150], Loss: 2.8778, Accuracy: 0.3603
Epoch [2/150], Loss: 1.7463, Accuracy: 0.3603
Epoch [3/150], Loss: 1.5434, Accuracy: 0.3603
Epoch [4/150], Loss: 1.4995, Accuracy: 0.3603
Epoch [5/150], Loss: 1.4531, Accuracy: 0.3824
Epoch [6/150], Loss: 1.4114, Accuracy: 0.3897
Epoch [7/150], Loss: 1.3575, Accuracy: 0.4044
Epoch [8/150], Loss: 1.2977, Accuracy: 0.4559
Epoch [9/150], Loss: 1.2946, Accuracy: 0.4926
Epoch [10/150], Loss: 1.2718, Accuracy: 0.5515
Epoch [11/150], Loss: 1.2560, Accuracy: 0.5294
Epoch [12/150], Loss: 1.2387, Accuracy: 0.6029
Epoch [13/150], Loss: 1.1953, Accuracy: 0.6029
Epoch [14/150], Loss: 1.2022, Accuracy: 0.6176
Epoch [15/150], Loss: 1.2186, Accuracy: 0.6324
Epoch [16/150], Loss: 1.2130, Accuracy: 0.6250
Epoch [17/150], Loss: 1.1928, Accuracy: 0.6250
Epoch [18/150], Loss: 1.1722, Accuracy: 0.6397
Epoch [19/150], Loss: 1.1701, Accuracy: 0.6324
Epoch [20/150], Loss: 

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader, random_split
from torchvision import models, transforms
import cv2
import os

# image transformations
transform = transforms.Compose([
    transforms.ToPILImage(),
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
])

# get all files from directories and subdirectories
def get_all_files(dir_path):
    all_files = []
    for root, _, files in os.walk(dir_path):
        for file in files:
            if file.endswith(('jpg', 'jpeg', 'png')):
                all_files.append(os.path.join(root, file))
    return all_files

# Define the Siamese Network
class SiameseNetwork(nn.Module):
    def __init__(self):
        super(SiameseNetwork, self).__init__()
        self.convnext = models.convnext_base(weights='IMAGENET1K_V1')

        # Freeze all layers except the classifier
        for param in self.convnext.parameters():
            param.requires_grad = False

        # Modify the fully connected layer
        self.convnext.classifier[2] = nn.Linear(self.convnext.classifier[2].in_features, 512)

    def forward_one(self, x):
        x = self.convnext(x)
        return x

    def forward(self, input1, input2):
        output1 = self.forward_one(input1)
        output2 = self.forward_one(input2)
        return output1, output2

# Define the dataset for image pairs
class ImagePairDataset(Dataset):
    def __init__(self, image_dir, shots_dir, transform=None):
        self.image_dir = image_dir
        self.shots_dir = shots_dir
        self.transform = transform
        self.image_pairs = []
        self.labels = []
        self._prepare_data()

    def _prepare_data(self):
        shots_files = get_all_files(self.shots_dir)
        image_files = get_all_files(self.image_dir)

        print(f"Number of shots: {len(shots_files)}")
        print(f"Number of images in anonymized dataset: {len(image_files)}")

        for shot_path in shots_files:
            for img_path in image_files:
                shot_id = os.path.basename(os.path.dirname(shot_path))
                img_id = os.path.basename(os.path.dirname(img_path))

                # Label 1 if they are from the same person, otherwise 0
                label = 1 if shot_id == img_id else 0
                self.image_pairs.append((shot_path, img_path))
                self.labels.append(label)

        print(f"Total image pairs created: {len(self.image_pairs)}")

    def __len__(self):
        return len(self.image_pairs)

    def __getitem__(self, idx):
        image1_path, image2_path = self.image_pairs[idx]
        image1 = cv2.imread(image1_path)
        image2 = cv2.imread(image2_path)
        image1 = cv2.cvtColor(image1, cv2.COLOR_BGR2RGB)
        image2 = cv2.cvtColor(image2, cv2.COLOR_BGR2RGB)

        if self.transform:
            image1 = self.transform(image1)
            image2 = self.transform(image2)

        label = torch.tensor(self.labels[idx], dtype=torch.float32)
        return image1, image2, label

# compute the Euclidean distance
def euclidean_distance(a, b):
    return F.pairwise_distance(a, b)

# Training
def train(model, dataloader, criterion, optimizer, device):
    model.train()
    running_loss = 0.0
    for image1, image2, label in dataloader:
        image1, image2, label = image1.to(device), image2.to(device), label.to(device)
        optimizer.zero_grad()
        output1, output2 = model(image1, image2)
        loss = criterion(euclidean_distance(output1, output2), label)
        loss.backward()
        optimizer.step()
        running_loss += loss.item()
    return running_loss / len(dataloader)

# Evaluation function
def evaluate(model, dataloader, threshold, device):
    model.eval()
    correct = 0
    total = 0
    with torch.no_grad():
        for image1, image2, label in dataloader:
            image1, image2, label = image1.to(device), image2.to(device), label.to(device)
            output1, output2 = model(image1, image2)
            distance = euclidean_distance(output1, output2)
            predictions = (distance < threshold).float()
            correct += (predictions == label).sum().item()
            total += label.size(0)
    return correct / total

# classify similarity of shots with anonymized dataset
def classify_similarity(model, shots_dir, image_dir, threshold, device):
    model.eval()
    transform = transforms.Compose([
        transforms.ToPILImage(),
        transforms.Resize((224, 224)),
        transforms.ToTensor(),
    ])

    shots_files = get_all_files(shots_dir)
    image_files = get_all_files(image_dir)
    results = []

    with torch.no_grad():
        for shot_path in shots_files:
            shot_id = os.path.basename(os.path.dirname(shot_path))
            shot_name = os.path.basename(shot_path)
            shot_image = cv2.imread(shot_path)
            shot_image = cv2.cvtColor(shot_image, cv2.COLOR_BGR2RGB)
            shot_image = transform(shot_image).unsqueeze(0).to(device)

            for img_path in image_files:
                img_id = os.path.basename(os.path.dirname(img_path))
                img_name = os.path.basename(img_path)
                img_image = cv2.imread(img_path)
                img_image = cv2.cvtColor(img_image, cv2.COLOR_BGR2RGB)
                img_image = transform(img_image).unsqueeze(0).to(device)
                # label = 1 if shot_image == image_files else 0
                label = 1 if shot_id == img_id else 0
                output1, output2 = model(shot_image, img_image)
                distance = euclidean_distance(output1, output2).item()
                similar = distance < threshold
                results.append((shot_name, img_name, distance, similar))

                # Print shot name, image name, and similarity status
                if similar:
                    print(f"Shot: {shot_name}, Image: {img_name}, Distance: {distance:.4f}, Similar: True, label: {label}")
                else:
                    print(f"Shot: {shot_name}, Image: {img_name}, Distance: {distance:.4f}, Similar: False, label: {label}")

    return results

# Main function
def main(image_dir, shots_dir, batch_size=16, num_epochs=500, learning_rate=0.001, threshold=6.0):
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    dataset = ImagePairDataset(image_dir, shots_dir, transform=transform)

    if len(dataset) == 0:
        print("No image pairs found. Please check the dataset paths and image files.")
        return

    train_size = int(0.8 * len(dataset))
    test_size = len(dataset) - train_size

    if train_size == 0 or test_size == 0:
        print(f"Train size: {train_size}, Test size: {test_size}. Not enough data to split.")
        return

    train_dataset, test_dataset = random_split(dataset, [train_size, test_size])
    train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
    test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

    model = SiameseNetwork().to(device)
    criterion = nn.BCEWithLogitsLoss()
    optimizer = torch.optim.Adam(model.convnext.classifier[2].parameters(), lr=learning_rate)

    for epoch in range(num_epochs):
        train_loss = train(model, train_loader, criterion, optimizer, device)
        test_accuracy = evaluate(model, test_loader, threshold, device)
        print(f"Epoch [{epoch+1}/{num_epochs}], Loss: {train_loss:.4f}, Accuracy: {test_accuracy:.4f}")

    print("Training complete")
    torch.save(model.state_dict(), "/content/drive/MyDrive/urv/distance_model_covnext.pt")
    # Final test evaluation
    final_test_accuracy = evaluate(model, test_loader, threshold, device)
    print(f"Final Test Accuracy: {final_test_accuracy:.4f}")

    # Classify similarity
    classify_similarity(model, shots_dir, image_dir, threshold, device)

# Example usage
image_dir = '/content/drive/MyDrive/urv/Images_For_The_Person_i_Want_To_Identify_In_Anonymized_Dataset_with_noise'
shots_dir = '/content/drive/MyDrive/urv/Images_For_The_Person_i_Want_To_Identify_In_Anonymized_Dataset_without_noise'
main(image_dir, shots_dir)


Number of shots: 26
Number of images in anonymized dataset: 26
Total image pairs created: 676
Epoch [1/500], Loss: 1.3513, Accuracy: 0.6544
Epoch [2/500], Loss: 1.0521, Accuracy: 0.6544
Epoch [3/500], Loss: 1.0096, Accuracy: 0.6544
Epoch [4/500], Loss: 0.9836, Accuracy: 0.6544
Epoch [5/500], Loss: 0.9713, Accuracy: 0.6544
Epoch [6/500], Loss: 0.9711, Accuracy: 0.6544
Epoch [7/500], Loss: 0.9285, Accuracy: 0.6544
Epoch [8/500], Loss: 0.9208, Accuracy: 0.6544
Epoch [9/500], Loss: 0.9352, Accuracy: 0.6544
Epoch [10/500], Loss: 0.9186, Accuracy: 0.6544
Epoch [11/500], Loss: 0.9120, Accuracy: 0.6544
Epoch [12/500], Loss: 0.8845, Accuracy: 0.6544
Epoch [13/500], Loss: 0.8996, Accuracy: 0.6544
Epoch [14/500], Loss: 0.8699, Accuracy: 0.6544
Epoch [15/500], Loss: 0.8581, Accuracy: 0.6544
Epoch [16/500], Loss: 0.8589, Accuracy: 0.6544
Epoch [17/500], Loss: 0.8902, Accuracy: 0.6544
